<a href="https://colab.research.google.com/github/Rig12332113/embedding_system_hw3/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ble_scan_connection.py


In [ ]:
from bluepy.btle import Peripheral, UUID
from bluepy.btle import Scanner, DefaultDelegate

class ScanDelegate(DefaultDelegate):
    def __init__(self):
        DefaultDelegate.__init__(self)
    def handleDiscovery(self, dev, isNewDev, isNewData):
        if isNewDev:
            print ("Discovered device", dev.addr)
        elif isNewData:
            print ("Received new data from", dev.addr)

class NotifyDelegate(DefaultDelegate):
    def __init__(self, params):
        DefaultDelegate.__init__(self)
    def handleNotification(self, cHandle, data):
        print("Notification from Handle: 0x" + format(cHandle, '02X'))
        print(data)

scanner = Scanner().withDelegate(ScanDelegate())
devices = scanner.scan(10.0)
n = 0
addr = []
for dev in devices:
    print("%d: Device %s (%s), RSSI:%d dB " % (n, dev.addr, dev.addrType, dev.rrssi))
    addr.append(dev.addr)
    n += 1
    for (adtype, desc, value) in dev.getScanData():
        print (" %s=%s " % (desc, value))

number = input("Enter your device number: ")
print("Device", number)
num = int(num)
print(addr[num])

print("Connecting...")
dev = Peripheral(addr[num], 'random')
print("Services...")
for svc in dev.services:
    print(str(svc))

try:
    testService = dev.getServiceByUUID(UUID(0xfff0))
    for ch in testService.getCharacteristics():
        print(str(ch))

    ch = dev.getCharacteristics(uuid=UUID(0xfff2))[0]
    print(ch)
    notify_handle = ch.getHandle()+1
    dev.writeCharacteristic(notify_handle, b"\x01\x00")
    dev.setDelegate(NotifyDelegate(ch.getHandle()))
    if (ch.supportsRead()):
        print(ch.read())
    while True:
        if dev.waitForNotifications(1.0):
            print("Received notification!")
            continue
        print(ch.read())
finally:
    dev.disconnect()